In [18]:
import datetime
import os

import constants

from azure.ai.ml import command, Input, Output
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential



In [19]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=constants.SUBSCRIPTION_ID,
    resource_group_name=constants.RESOURCE_GROUP_NAME,
    workspace_name=constants.WORKSPACE_NAME,
)

In [20]:
dict1 = {
    "TXN": {
        "name": [
            "Texas Instruments Incorporated (<b>TXN</b>) May Have A Gold Mine.",
            "Unusual Call Option Trade in Texas Instruments (<b>TXN</b>) Worth $1,995.00K",
            "What&#39;s in the Offing for Texas Instruments (<b>TXN</b>) in Q2 Earnings?",
            "Mizuho Maintains Texas Instruments (<b>TXN</b>) Neutral Recommendation",
            "Texas Instruments Unusual Options Activity",
            "Texas Instruments Incorporated (<b>TXN</b>) Stock Records 2.47% Quarterly Movement",
            "Selling Your Texas Instruments Incorporated (NASDAQ: <b>TXN</b>) Stock? Here\u2019s What You Need To Know",
            "Texas Instruments board declares third quarter 2023 quarterly dividend"
        ],
        "description": [
            "Texas Instruments Incorporated (NASDAQ:TXN) price on Friday, July 21, rose 2.45% above its previous day\u2019s close as an upside momentum from buyers pushed the stock\u2019s value to $184.32. A look at the stock\u2019s price movement,",
            "On July 21, 2023 at 15:50:03 ET an unusually large $1,995.00K block of Call contracts in Texas Instruments (<b>TXN</b>) was bought, with a strike price of $200.00 / share, expiring in 182 day(s) (on January 19, 2024). Fintel tracks all large options trades ...",
            "Texas Instruments&#39; (TXN) second-quarter 2022 results are likely to reflect the adverse effects of inventory reduction on Analog and Embedded Processing segments.",
            "Fintel reports that on July 20, 2023, Mizuho maintained coverage of Texas Instruments (NASDAQ:<b>TXN</b>) with a Neutral recommendation. Analyst Price Forecast Suggests 2.51% Upside As of July 6, 2023, the average one-year price target for Texas Instruments is ...",
            "In terms of liquidity and interest, the mean open interest for Texas Instruments options trades today is 1554.12 with a total volume of 1,617.00.",
            "The stock of Texas Instruments Incorporated (TXN) has gone up by 2.21% for the week, with a 3.55% rise in the past month and a 2.47% rise in the past quarter. The volatility ratio for the week is 1.84%,",
            "Texas Instruments Incorporated (NASDAQ:TXN) shares, rose in value on Friday, July 07, with the stock price up by 0.48% to the previous day\u2019s close as strong demand from buyers drove the stock to $175.",
            "The board of directors of Texas Instruments Incorporated (NASDAQ:TXN) today declared a quarterly cash dividend of $1.24 per share of common stock, payable August 15, 2023, to"
        ],
        "datePublished": [
            "2023-07-22T15:36:00.0000000Z",
            "2023-07-22T15:36:00.0000000Z",
            "2023-07-20T14:53:00.0000000Z",
            "2023-07-21T15:44:00.0000000Z",
            "2023-07-20T20:45:00.0000000Z",
            "2023-07-18T07:04:00.0000000Z",
            "2023-07-07T20:37:00.0000000Z",
            "2023-07-20T21:28:00.0000000Z"
        ],
        "url": [
            "https://stocksregister.com/2023/07/22/texas-instruments-incorporated-txn-may-have-a-gold-mine/",
            "https://www.msn.com/en-us/money/companies/unusual-call-option-trade-in-texas-instruments-txn-worth-199500k/ar-AA1ecS2j",
            "https://finance.yahoo.com/news/whats-offing-texas-instruments-txn-145300746.html",
            "https://www.msn.com/en-us/money/companies/mizuho-maintains-texas-instruments-txn-neutral-recommendation/ar-AA1eaZS9",
            "https://www.benzinga.com/markets/options/23/07/33312387/texas-instruments-unusual-options-activity",
            "https://newsheater.com/2023/07/18/texas-instruments-incorporated-txn-stock-records-2-47-quarterly-movement/",
            "https://stocksregister.com/2023/07/07/selling-your-texas-instruments-incorporated-nasdaq-txn-stock-heres-what-you-need-to-know/",
            "https://www.benzinga.com/pressreleases/23/07/n33313116/texas-instruments-board-declares-third-quarter-2023-quarterly-dividend"
        ],
        "texts": [
            "Texas Instruments Incorporated (NASDAQ:TXN) price on Friday, July 21, rose 2.45% above its previous day\u2019s close as an upside momentum from buyers pushed the stock\u2019s value to $184.32. A look at the stock\u2019s price movement, the close in the last trading session was $179.92, moving within a range at $181.355 and $185.57. The PE ratio in trailing twelve months stood at 20.68. Turning to its 52-week performance, $186.30 and $145.97 were the 52-week high and 52-week low respectively. Overall, TXN moved 8.12% over the past month. Top 5 AI Stocks to Buy for 2023 The artificial intelligence (AI) revolution is",
            "",
            "Texas Instruments Incorporated TXN is scheduled to report second-quarter 2023 results on Jul 25. For second-quarter 2023, Texas Instruments expects revenues between $4.17 billion and $4.53 billion. The Zacks Consensus Estimate is pegged at $4.36 billion, suggesting a decline of 16.4% from the year-ago quarter\u2019s reported figure. Management expects earnings of $1.62-$1.88 per share for the quarter under review. The consensus mark is pegged at $1.76 per share, indicating a fall of 28.2% from the prior-year quarter\u2019s reported figure. The estimate has remained unchanged over the past 30 days. TXN\u2019s earnings surpassed the Zacks Consensus Estimate in all the trailing",
            "",
            "A whale with a lot of money to spend has taken a noticeably bearish stance on Texas Instruments. Looking at options history for Texas Instruments TXN we detected 14 strange trades. If we consider the specifics of each trade, it is accurate to state that 35% of the investors opened trades with bullish expectations and 64% with bearish. From the overall spotted trades, 6 are puts, for a total amount of $667,765 and 8, calls, for a total amount of $379,357. What's The Price Target? Taking into account the Volume and Open Interest on these contracts, it appears that whales",
            "The stock of Texas Instruments Incorporated (TXN) has gone up by 2.21% for the week, with a 3.55% rise in the past month and a 2.47% rise in the past quarter. The volatility ratio for the week is 1.84%, and the volatility levels for the past 30 days are 1.92% for TXN. The simple moving average for the last 20 days is 4.02% for TXN stock, with a simple moving average of 6.86% for the last 200 days. Is It Worth Investing in Texas Instruments Incorporated (NASDAQ: TXN) Right Now? Texas Instruments Incorporated (NASDAQ: TXN) has a higher price-to-earnings ratio",
            "Texas Instruments Incorporated (NASDAQ:TXN) shares, rose in value on Friday, July 07, with the stock price up by 0.48% to the previous day\u2019s close as strong demand from buyers drove the stock to $175.66. Actively observing the price movement in the recent trading, the stock is buoying the session at $174.82, falling within a range of $173.08 and $175.48. The value of beta (5-year monthly) is 1.02 whereas the PE ratio is 19.62 over 12-month period. Referring to stock\u2019s 52-week performance, its high was $186.30, and the low was $145.97. On the whole, TXN has fluctuated by 3.11% over the",
            "DALLAS, July 20, 2023 /PRNewswire/ -- The board of directors of Texas Instruments Incorporated TXN today declared a quarterly cash dividend of $1.24 per share of common stock, payable August 15, 2023, to stockholders of record on July 31, 2023. About Texas Instruments Texas Instruments Incorporated TXN is a global semiconductor company that designs, manufactures, tests and sells analog and embedded processing chips for markets such as industrial, automotive, personal electronics, communications equipment and enterprise systems. Our passion to create a better world by making electronics more affordable through semiconductors is alive today, as each generation of innovation builds upon"
        ],
    }
}

dict2 = {
    "DOCN": {
        "name": [
            "DigialOCean did something",
            "DigialOCean did soemthing"
        ],
        "description": [
            "DigialOCean did something",
            "DigialOCean did soemthing"
        ],
        "datePublished": [
            "DigialOCean did something",
            "DigialOCean did soemthing"
        ],
        "url": [
            "DigialOCean did something",
            "DigialOCean did soemthing"
        ],
        "texts": [
            "DigialOCean did something",
            "DigialOCean did soemthing"
        ],
    }
}

In [21]:
all_dict = {}

dict_list = [dict1, dict2]
for item in dict_list: 
    all_dict.update(item)

In [22]:
%%writefile components/prep.py 

from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from pathlib import Path
import datetime
import json

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")
secret_client = SecretClient(vault_url="https://mlgroup.vault.azure.net/", credential=credential)

import argparse

parser = argparse.ArgumentParser("prep")
parser.add_argument("--blob_storage", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--prep_output")
args = parser.parse_args()

# log in to the Blob Service Client
blob_storage = args.blob_storage
blob_storage_key = secret_client.get_secret("blob-storage-key")
blob_service_client = BlobServiceClient(blob_storage, account_key=blob_storage_key.value)

# connect to the container 
container_client = blob_service_client.get_container_client(container="stock-news-json") 

# list and download all currently available blobs
blob_list = container_client.list_blobs()
print(f"Blob from: {blob_storage} has these blobs today: {blob_list}")

# get the timestamp with the current day 
current_day_date = datetime.datetime.today().isoformat()[:10]

blobs_to_use = [blob.name for blob in blob_list if current_day_date in blob.name]
for blob in blobs_to_use:
      print(f"Downloading blob: {blob}")
      blob_client = blob_service_client.get_blob_client(container="stock-news-json", blob=blob)
      with open(blob, mode="wb") as sample_blob:
            download_stream = blob_client.download_blob()
            sample_blob.write(download_stream.readall())

all_data_dict = {}
for json_file in blobs_to_use:
      with open(json_file,"r+") as file:
      # First we load existing data into a dict.
            file_data = json.load(file)
            all_data_dict.update(file_data)
            
with open((Path(args.prep_output) / "merged_stock_news.json"), "w") as file:
      file.write(json.dumps(all_data_dict, indent=4))

# this is a comments

Overwriting components/prep.py


In [23]:
%%writefile components/classify.py

from pathlib import Path
import argparse
import json
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification

parser = argparse.ArgumentParser()
parser.add_argument("--classify_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--classify_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

# download distilbert model from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("KernAI/stock-news-destilbert")
model = AutoModelForSequenceClassification.from_pretrained("KernAI/stock-news-destilbert")

# retriev the list of blobs from the current day - input is a .txt file
with open(os.path.join(args.classify_input, "merged_stock_news.json"), "r") as f:
      data = json.load(f)

# get a list of all tickers in the data   
tickers = list(data.keys())
for ticker in tickers:
      texts = data[ticker]["texts"]
      sentiments = []
      for text in texts: 
            tokenized_text = tokenizer(
                  text,
                  truncation=True,
                  is_split_into_words=False,
                  return_tensors="pt"
            )

            outputs = model(tokenized_text["input_ids"])
            outputs_logits = outputs.logits.argmax(1)

            mapping = {0: 'neutral', 1: 'negative', 2: 'positive'}
            predicted_label = mapping[int(outputs_logits[0])]
            sentiments.append(predicted_label)

      # add the sentiments to the data
      data[ticker]["sentiments"] = sentiments

# overwrite old files with new files containing the sentiment
with open((Path(args.classify_output) / "merged_stock_news.json"), "w") as f:
      json.dump(data, f)

Overwriting components/classify.py


In [24]:
%%writefile components/summarize.py

from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

from pathlib import Path
import datetime 
import argparse
import json
import os

from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")
secret_client = SecretClient(vault_url="https://mlgroup.vault.azure.net/", credential=credential)

parser = argparse.ArgumentParser()
parser.add_argument("--summarize_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--summarize_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

# load the model and the tokenizer
tokenizer = PegasusTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = PegasusForConditionalGeneration.from_pretrained("human-centered-summarization/financial-summarization-pegasus") 

# retriev the list of blobs from the current day - input is a .txt file
with open(os.path.join(args.summarize_input, "merged_stock_news.json"), "r") as f:
      data = json.load(f)

tickers = list(data.keys())
for ticker in tickers:
      texts = data[ticker]["texts"]

      summaries = []
      for text in texts: 
            # Tokenize our text
            # If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
            input_ids = tokenizer(text, return_tensors="pt").input_ids

            # Generate the output (Here, we use beam search but you can also use any other strategy you like)
            output = model.generate(
                  input_ids, 
                  max_length=32, 
                  num_beams=5, 
                  early_stopping=True
            )

            # Finally, we can print the generated summary
            summaries.append(tokenizer.decode(output[0], skip_special_tokens=True))

      # add the sentiments to the data
      data[ticker]["summaries"] = summaries
      
data = json.dumps(data)

# connect and authenticate to the blob client
account_url = "https://mlstorageleo.blob.core.windows.net"
file_name = f"processed-stock-news-{datetime.datetime.today().isoformat()[:10]}.json"

# Create the BlobServiceClient object
blob_storage_key = secret_client.get_secret("blob-storage-key")
blob_service_client = BlobServiceClient(account_url, credential=blob_storage_key.value)
blob_client = blob_service_client.get_blob_client(container="processed-stock-news-json", blob=file_name)
blob_client.upload_blob(data)

# overwrite old files with new files containing the sentiment
with open((Path(args.summarize_output) / "merged_stock_news.json"), "w") as f:
      json.dump(data, f)

Overwriting components/summarize.py


In [25]:
%%writefile dependencies/conda.yml
name: stock-analysis-env
channels:
  - conda-forge
dependencies:
  - python=3.9
  - pip
  - pip:
    - azure-storage-blob
    - azure-identity
    - azure-keyvault
    - transformers
    - torch
    - sentencepiece
    - numpy

Overwriting dependencies/conda.yml


In [26]:
custom_env_name = "stock-analysis-env"
version = "1.6"

try:    
    pipeline_job_env = ml_client.environments.get(custom_env_name, version=version)

except:
    pipeline_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for stock analysis pipeline",
        conda_file=os.path.join("dependencies", "conda.yml"),
        image="mcr.microsoft.com/azureml/curated/python-sdk-v2:4",
        version=version,
    )
    pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )

In [27]:
print(["TXT"] * 7)

['TXT', 'TXT', 'TXT', 'TXT', 'TXT', 'TXT', 'TXT']


In [28]:
data_type = AssetTypes.URI_FOLDER
path = "azureml://datastores/stocknewsjson/stock-news-json"
input_mode = InputOutputModes.RO_MOUNT
output_mode = InputOutputModes.RW_MOUNT

In [29]:
data_prep_component = command(
    name="data_prep",
    display_name="Finding out which blobs to actually use",
    description="Loads files from Azure Blob Storage from todays ",
    inputs={
        "blob_storage": Input(mode=InputOutputModes.DIRECT)
    },
    outputs={
        "prep_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/prep.py",
    command="python prep.py --blob_storage ${{inputs.blob_storage}} --prep_output ${{outputs.prep_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [30]:
classify_component = command(
    name="data_prep",
    display_name="Classify the sentiments of todays stock news",
    description="Loads data via AlphaVantage API input, preps data and stores to as data asset",
    inputs={
        "classify_input": Input(type=data_type, mode=input_mode), 
    },
    outputs={
        "classify_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/classify.py",
    command="python classify.py --classify_input ${{inputs.classify_input}} --classify_output ${{outputs.classify_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [31]:
summarize_component = command(
    name="data_prep",
    display_name="Summarize the news",
    description="Loads data via AlphaVantage API input, preps data and stores to as data asset",
    inputs={
        "summarize_input": Input(type=data_type, mode=input_mode),
    },
    outputs={
        "summarize_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/summarize.py",
    command="python summarize.py --summarize_input ${{inputs.summarize_input}} --summarize_output ${{outputs.summarize_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [32]:
from azure.ai.ml.dsl import pipeline

@pipeline(compute="ava")
def stock_news_pipeline():

    data_prep_job = data_prep_component(
        blob_storage="https://mlstorageleo.blob.core.windows.net/"
    )
    classify_job = classify_component(
        classify_input=data_prep_job.outputs.prep_output

    ) # feed putput of previous step into the training job
    summarize_job = summarize_component(
        summarize_input = classify_job.outputs.classify_output
    )

    return {"processed_file": summarize_job.outputs.summarize_output}

pipeline_job = stock_news_pipeline()

# set pipeline level compute
pipeline_job.settings.default_compute = "ava"
pipeline_job.settings.reuse_component = "false"

In [33]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="stock-news-analysis-pipeline"
)
pipeline_job

Uploading summarize.py (< 1 MB): 100%|##########| 2.98k/2.98k [00:00<00:00, 38.6kB/s]




Experiment,Name,Type,Status,Details Page
stock-news-analysis-pipeline,teal_wheel_ghmct9469p,pipeline,Preparing,Link to Azure Machine Learning studio


In [34]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: teal_wheel_ghmct9469p
Web View: https://ml.azure.com/runs/teal_wheel_ghmct9469p?wsid=/subscriptions/5a361d37-b562-4eee-981b-0936493063e9/resourcegroups/mlgroup/workspaces/mlworkspace

Streaming logs/azureml/executionlogs.txt

[2023-07-22 19:45:58Z] Submitting 1 runs, first five are: c870d7d7:b8999d93-35d8-47d4-892f-ede848eb69d0
[2023-07-22 19:46:28Z] Completing processing run id b8999d93-35d8-47d4-892f-ede848eb69d0.
[2023-07-22 19:46:28Z] Submitting 1 runs, first five are: efba5f2e:f59b90ce-fe5e-48c9-81fe-0246663824d1
[2023-07-22 19:48:08Z] Completing processing run id f59b90ce-fe5e-48c9-81fe-0246663824d1.
[2023-07-22 19:48:08Z] Submitting 1 runs, first five are: 78f4424f:7ccb0bad-dcae-46ce-a0b8-76cd10a1e7eb
[2023-07-22 20:33:12Z] Completing processing run id 7ccb0bad-dcae-46ce-a0b8-76cd10a1e7eb.

Execution Summary
RunId: teal_wheel_ghmct9469p
Web View: https://ml.azure.com/runs/teal_wheel_ghmct9469p?wsid=/subscriptions/5a361d37-b562-4eee-981b-0936493063e9/resourcegroups/mlgroup